In [1]:
import os
import numpy as np
import glob
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Add, Activation
from tensorflow.keras.layers import  Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
np.random.seed(406)

base_dir = "/content/drive/My Drive/kvasir-dataset-v2"
classes = ['dyed-lifted-polyps',           'dyed-resection-margins',           'esophagitis',           'normal-cecum',           'normal-pylorus',           'normal-z-line',           'polyps',           'ulcerative-colitis']

for cl in classes:
    
    img_path = os.path.join(base_dir, cl)
    images = glob.glob(img_path + '/*.jpg')
    print("{}: {} Images".format(cl, len(images)))
    
    np.random.shuffle(images)    # Shuffle the images randomly
    
    train_list, val_list, test_list = images[:round(len(images)*0.7)], images[round(len(images)*0.7):round(len(images)*0.85)], images[round(len(images)*0.85):]

    for image in train_list:
        if not os.path.exists(os.path.join(base_dir, 'train', cl)):
            os.makedirs(os.path.join(base_dir, 'train', cl))
        shutil.move(image, os.path.join(base_dir, 'train', cl))

    for image in val_list:
        if not os.path.exists(os.path.join(base_dir, 'val', cl)):
            os.makedirs(os.path.join(base_dir, 'val', cl))
        shutil.move(image, os.path.join(base_dir, 'val', cl))
        
    for image in test_list:
        if not os.path.exists(os.path.join(base_dir, 'test', cl)):
            os.makedirs(os.path.join(base_dir, 'test', cl))
        shutil.move(image, os.path.join(base_dir, 'test', cl))


dyed-lifted-polyps: 0 Images
dyed-resection-margins: 0 Images
esophagitis: 0 Images
normal-cecum: 0 Images
normal-pylorus: 0 Images
normal-z-line: 0 Images
polyps: 0 Images
ulcerative-colitis: 0 Images


In [4]:
input_shape = (224, 224, 3)
num_classes = len(classes)

input_layer = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

def mixup_data(x, y, alpha=0.2):
    x = np.array(x)  # Convert x to array
    y = np.array(y)  # Convert y to array
    
    batch_size = x.shape[0]
    lam = np.random.beta(alpha, alpha, batch_size)
    index = np.random.permutation(batch_size)
    mixed_x = lam.reshape(batch_size, 1, 1, 1) * x + (1 - lam).reshape(batch_size, 1, 1, 1) * x[index, :]
    mixed_y = lam.reshape(batch_size, 1) * y + (1 - lam).reshape(batch_size, 1) * y[index, :]
    
    return mixed_x, mixed_y

def cutmix_data(x, y, alpha=1.0):
    batch_size = x.shape[0]
    lam = np.random.beta(alpha, alpha, batch_size)
    index = np.random.permutation(batch_size)
    bbx1, bby1, bbx2, bby2 = get_random_bbox(x.shape[1], x.shape[2], lam)
    mixed_x = x.copy()
    mixed_x[:, bbx1:bbx2, bby1:bby2, :] = x[index, bbx1:bbx2, bby1:bby2, :]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.shape[1] * x.shape[2]))
    mixed_y = lam * y + (1 - lam) * y[index, :]
    return mixed_x, mixed_y

def get_random_bbox(width, height, lam):
    cut_rat = np.sqrt(1.0 - lam)
    cut_w = np.int(width * cut_rat)
    cut_h = np.int(height * cut_rat)
    cx = np.random.randint(width)
    cy = np.random.randint(height)
    bbx1 = np.clip(cx - cut_w // 2, 0, width)
    bby1 = np.clip(cy - cut_h // 2, 0, height)
    bbx2 = np.clip(cx + cut_w // 2, 0, width)
    bby2 = np.clip(cy + cut_h // 2, 0, height)
    return bbx1, bby1, bbx2, bby2

def mixup_loss(y_true, y_pred):
    lam = K.variable(1.0)
    return lam * K.categorical_crossentropy(y_true, y_pred) + (1 - lam) * K.categorical_crossentropy(y_true[::-1], y_pred[::-1])

train_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=lambda x: x * 1.0  # Convert x to a numerical array
)
test_data_generator = ImageDataGenerator(rescale=1.0/255.0)


batch_size = 32
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

train_data = train_data_generator.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)
test_data = test_data_generator.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

epochs = 3

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // batch_size,
    epochs=epochs,
    validation_data=test_data,
    validation_steps=test_data.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

loss, accuracy = model.evaluate(test_data)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Found 5608 images belonging to 8 classes.
Found 1202 images belonging to 8 classes.
Epoch 1/3
175/175 [==============================] - 2735s 16s/step - loss: 1.0762 - accuracy: 0.5273 - val_loss: 0.8167 - val_accuracy: 0.5997
Epoch 2/3
175/175 [==============================] - 917s 5s/step - loss: 0.8061 - accuracy: 0.6273 - val_loss: 0.6420 - val_accuracy: 0.7010
Epoch 3/3
38/38 [==============================] - 60s 2s/step - loss: 0.6528 - accuracy: 0.7072
Test Loss: 0.6528
Test Accuracy: 0.7072
